In [1]:
# RF RandomForest
# ET ExtraTree
# GB GradientBoost
# HGB HistogramGradientBoost 작년까지는 beta version

## 트리의 앙상블

In [2]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델들 입니다.
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어졌습니다.
# 앙상블 모델들..
# - 랜덤포레스트(Random Forest)
# - 엑스트라 트리(Extra Trees)
# - 그래디언트 부스팅(Gradient Boosting)
# - 히스토그램 기반 그래디언트 부스팅(Histogram-base Gradient Boosting)

In [3]:
# - 여러모델을 써야하는 이유  : 비교대상이 필요하고 최적의 모델이란것을 증명해야하기위해

## 랜덤포레스트(Random Forest)

In [4]:
## 앙상블 모델 중에 가장 대표격 모델로 사용됨
# - 안정적인 성능으로 널리 사용됨
# - 앙상블 모델 중에 가장 먼저 시도하는 모델이라고 보면 됩니다.
# - 훈련데이터에서 과대적합되는 것을 막아줍니다
# - 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

### 학습 개념
# - 결정트리 하나하나를 랜덤하게 만들어서 숲을 만든다고 보시면 됩니다.
# - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후
# - 다시 원본 훈련데이터에 반환을 합니다.
# - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음
# (중복을 허용함)

### 부트스트랩 샘플(Bootstrap Sample)
# - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
# - 샘플 추출 방식
# - 1. 원본에서 랜덤 샘플 추출
# - 2. 훈련 이후 사용이 끝나면 원본에 반환
# - 3. 다시 원본에서 샘플 추출, 이때 중복 값 추출 될 수도 있음
# - 위 순서를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용하고 있음

### *** 랜덤포레스트는 교차검증을 허용 합니다 ***


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
### 08_wine.csv 읽어들이고 훈련 및 테스트 데이터까지 생성해 주세요 ...

wine = pd.read_csv('./data/08_wine.csv')
wine.head()

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = \
    train_test_split(data,target, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


## 훈련모델 생성하기

In [7]:
# 랜덤포레스트 클래스(모델) : RandomForestClassifire
# 교차검증 : cross_validate()
# 교차검증 후 훈련검증결과와 테스트검증결과 확인하기
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold 
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(RandomForestClassifier(random_state=42),train_input, train_target, cv=splitter)
print(scores)


{'fit_time': array([0.26483655, 0.29620576, 0.25982141, 0.2443409 , 0.28933406,
       0.32096195, 0.27701807, 0.26013207, 0.26331234, 0.2709012 ]), 'score_time': array([0.0149684 , 0.01491737, 0.01501107, 0.0182097 , 0.02890229,
       0.01564622, 0.01368499, 0.01955104, 0.01113677, 0.01473427]), 'test_score': array([0.90778689, 0.87295082, 0.91581109, 0.88090349, 0.91786448,
       0.90349076, 0.8788501 , 0.87474333, 0.87063655, 0.89117043])}


In [8]:
print('최종 score = ', np.mean(scores['test_score']))

최종 score =  0.8914207930790724


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# 랜덤포레스트 객체생성 : 코어 모두 사용
rf = RandomForestClassifier(n_jobs = -1 , random_state = 42)

# 교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)

# 최종 훈련평가 결과 및 검증결과
scores
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683


In [10]:
rf.fit(train_input, train_target)
# 특성 중요도 조회하기
rf.feature_importances_

array([0.23155241, 0.49706658, 0.27138101])

In [11]:
### oob : 기능 사용
# - 훈련에 참여하지 못한 잔여샘플 사용하는 기능
# 기본은 사용안함,
# 기본은 사용안함
# 훈련한번더하고 검증까지
rf = RandomForestClassifier(oob_score = True, n_jobs = -1 , random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8981937602627258


## 엑스트라 트리(Extra Tree)

In [12]:
# - 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련함
# - 랜덤포레스트와의 차이점
# - 부트스트랩 샘플링을 지원하지 않음
# - 훈련데이터 전체를 이용하여 결정트리를 생성
# - 무작위로 트리를 분리함
# - 사용되는 속성 : splitter = "random" 무작위속성
# - 장점
# : 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
# : 특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정 트리를 생성하기 때문에
#   속도가 느린 반면에
# - 엑스트라트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점있음

In [13]:
### 사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreesClassifier

### 코어 전체사용 : train 및 test 결과값 출력 ..
### 교차검증 결과만 train 및 test 결과 확인해 주세요

train_input, test_input, train_target, test_target = \
    train_test_split(data,target, random_state=42)

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
# 교차검증 진행
# - return_train_score : 검증결과 반환받기
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)

## 최종 및 훈련평가 결과 및 검증결과
scores
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683

훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804


In [14]:
et.fit(train_input,train_target)
et.feature_importances_
# array([0.23155241, 0.49706658, 0.27138101]) 랜덤포레스트

array([0.20702369, 0.51313261, 0.2798437 ])

## 그래디언트 부스팅(Gradient Boosting)

In [15]:
# 깊이(max_depth)가 얖은 결정트리를 사용함
# - 기본적으로 max_depth = 3 을 사용
# - 결정트리는 100개 사용
### *** 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 ***
# 기존 훈련모델의 오차를 많이 보완해 줍니다
# 성능 향상을 위한 모델로 주로 사용됩니다.
# 과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강합니다.

# 성능향상 테스트 방법
# - 결정트리의 갯수를 조절하면서 테스트 짆냉
# - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
# - : 기본 학습률은 0.1


# 단점
# - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에
# - 훈련 속도는 느림
# - 이런 느린 속도를 개선한 모델이
#  '히스토그램 기반 그래디언트 부스팅' 모델임

# 어느때 어떻게 써야될지 생각하는게 어렵다

## 그래디언트 부스팅 모델 생성

In [16]:
### 사용하는 클래스(모델) : GradientBoostingClassifier
# 객체 생성시 아무것도 안주고 seed값만 줍니다.
# 교차검증시에는 train, test 결과값 출력 합니다.
train_input, test_input, train_target, test_target = \
    train_test_split(data,target, random_state=42)

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

# (해석 ) 과대적합이 줄었다 => 모델이 일반화 되었다, 성능은 감소하였다 0.1이상 격차가 나면 선택하면 안된다
# 우선순위 : 일반화된거 선택해야함
# 성능향상시킬 필요는 있음

## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683
## 엑스트라트리 
# 훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804

훈련 score =  0.8894704231708938 테스트 score=  0.8715107671247301


In [17]:
gb.fit(train_input, train_target)
print(et.feature_importances_)
gb.score(test_input, test_target)

[0.20702369 0.51313261 0.2798437 ]


0.8578461538461538

## 학습률 적용하기

In [18]:
#   학습률이 커지면 트리 보정을 강하게 하기 때문에,
#   복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게 된다.
# 학습률 : learning_rate = 0.1 기본값...
# 값이 낮을수록 과대적합, 과소적합을 줄여줄수있다
# 값을 너무 올리면 복잡도가 엄청나게 강해진다 => 과대적합가능성이 오른다 일반화 떨어진다

In [19]:
# 훈련 반복횟수 n_estimator
train_input, test_input, train_target, test_target = \
    train_test_split(data,target, random_state=42)

from sklearn.ensemble import GradientBoostingClassifier
gd = GradientBoostingClassifier(n_estimators = 100,
                                learning_rate = 0.1,
                                random_state = 42)
scores = cross_validate(gd, train_input, train_target, return_train_score = True, n_jobs = -1)

print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

# 학습률은 안건드리는게 좋다 올리면 과대적합이 발생할 가능성이 늘어난다
# 과소적합일때는 올려서 테스트할 필요는 있다


## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683
## 엑스트라트리 
# 훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804

훈련 score =  0.8894704231708938 테스트 score=  0.8715107671247301


## 히스토그램 기반 그래디언트 부스팅
##### -Histogram-base Gradient Boosting

In [20]:
## 사용하는 클래스(모델) : HistGradientBoostingClassifier()
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683
## 엑스트라트리 
# 훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804
## 그래디언트
# 훈련 score =  0.8894704231708938 테스트 score=  0.8715107671247301
## HGB
# 훈련 score =  0.9380129799494501 테스트 score=  0.8805410414363187

## (해석) 0.05정도의 차이니까 HGB선택한다
# 좋은모델정도?로 평가한다 굉장히 주관적이다

훈련 score =  0.9380129799494501 테스트 score=  0.8805410414363187


In [22]:
hgb.fit(train_input, train_target)
hgb.score(test_input, test_target)

0.8584615384615385

## 사이킷런 이외 다른 패키지에서 지원하는
## 히스토그램 기반 그래디언트 부스팅 기능 모델들

## XGBoost

In [28]:
# 아나콘다 사용하시는 분들 : conda install -c conda-forge xgboost

In [25]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method="hist", random_state=42)

scores = cross_validate(xgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))
## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683
## 엑스트라트리 
# 훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804
## 그래디언트
# 훈련 score =  0.8894704231708938 테스트 score=  0.8715107671247301
## HGB
# 훈련 score =  0.9380129799494501 테스트 score=  0.8805410414363187
## xgb
# 훈련 score =  0.9614122399872658 테스트 score=  0.8834151529510873

C:\Users\admin\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


훈련 score =  0.9614122399872658 테스트 score=  0.8834151529510873


## LightGBM

In [26]:
# - 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
# - 훈련 속도가 매우 빠름
# - 최신 기술을 많이 적용하고 있어서, 인기가 올라가고 있음
# - 

In [24]:
# 아나콘다 사용하시는 분들 : conda install -c conda-forge lightgbm
# 파이썬 기반 사용하시는분들 pip install lightgbm
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state=42)

scores = cross_validate(lgbm, train_input, train_target, return_train_score = True, n_jobs = -1)
print('훈련 score = ',np.mean(scores['train_score']),'테스트 score= ' ,np.mean(scores['test_score']))

## 랜포
# 훈련 score =  0.997844759088341 테스트 score=  0.8914208392565683
## 엑스트라트리 
# 훈련 score =  0.997844759088341 테스트 score=  0.8903937240035804
## 그래디언트
# 훈련 score =  0.8894704231708938 테스트 score=  0.8715107671247301

## HGB
# 훈련 score =  0.9380129799494501 테스트 score=  0.8805410414363187
## xgb
# 훈련 score =  0.9614122399872658 테스트 score=  0.8834151529510873
# lightgbm
# 훈련 score =  0.9413484712095832 테스트 score=  0.8846461327857632

훈련 score =  0.9413484712095832 테스트 score=  0.8846461327857632


In [27]:
# 내일은 비정형 데이터 머신러닝 !